In [1]:
!pip install torch_geometric

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 8.5 MB/s  0:00:00


In [2]:
import torch
from torch_geometric.data import Data

### GET STARTED 

#### Data Handling of Graphs

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
# data.edge_index : Graph connectivity in COO format with shape [2, num_edges]
# 첫 번재 행: 엣지의 출발 노드(source node), 두 번째 행: 엣지의 도착 노드(target node)
# 1열: 노드 0 -> 노드 1로 가는 엣지, 2열: 노드 1 -> 노드 0로 가는 엣지, 
# 3열: 노드 1 -> 노드 2로 가는 엣지, 4열: 노드 2 -> 노드 1로 가는 엣지

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# data.x : Node feature matrix with shape [num_nodes, num_node_features]

data = Data(x=x, edge_index=edge_index)
# x와 edge_index 모두 tesor 형태여야 함
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [ ]:
data.is_undirected()  # True
data.is_directed()  # False

False

In [ ]:
Data(edge_index=[2, 4], x=[3, 1])
# tensor가 아닌 list를 넣음. -> 즉, 길이가 2인 list로 출력됨.
# 같은 이름으로 변수를 넣었다고 해도 tensor가 아니라면, 
# PyG가 그래프 구조로 인식하지 못함. (파이선 리스트로 인식)
# Data(edge_index=torch.tensor([2,4]), x=torch.tensor([3,1]))로 하면 tensor로 인식.

Data(x=[2], edge_index=[2])

In [84]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index.t().contiguous())

print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [85]:
Data(edge_index=[2, 4], x=[3, 1])

Data(x=[2], edge_index=[2])

In [87]:
data.validate(raise_on_error=True)
# Data 객체가 GNN 연산에 쓸 수 있는 정상적인 그래프인지 검사

# (1) x, edge_index의 shape이 올바른지, 
# (2) edge_index가 올바른 범위([0, num_nodes-1])의 노드 인덱스인지, 
# (3) self-loops나 isolated nodes가 있는지 등등

True

In [18]:
print(data.keys())
print(data['x'])
print(data['edge_index'])
print("\n")

for key, item in data:
    print(f'{key}: {item}')

['edge_index', 'x']
tensor([[-1.],
        [ 0.],
        [ 1.]])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


x: tensor([[-1.],
        [ 0.],
        [ 1.]])
edge_index: tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [92]:
'edge_attr' in data  # False
data.num_nodes  # 3
data.num_edges  # 4
data.num_node_features  # 1
data.num_edge_features  # 0

data.has_isolated_nodes()  # False  
# 어떤 엣지에도 연결되지 않은 노드가 있는지
# 전처리 시 사용: 데이터셋에 불필요한 노드 확인 후 제거
data.has_self_loops()  # False   
# 자기 자신으로 향하는 엣지가 있는지, 즉 (i -> i) 형태의 엣지
# 전처리 시 사용: 이중 엣지 제거

data.is_directed()  # False
data.to_dict()  # {'x': ..., 'edge_index': ...}

{'x': tensor([[-1.],
         [ 0.],
         [ 1.]]),
 'edge_index': tensor([[0, 1, 1, 2],
         [1, 0, 2, 1]])}

In [ ]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

#### Common Benchmark Datasets

In [ ]:
from torch_geometric.datasets import TUDataset
# 여러 benchmark 데이터셋 존재

ENZYMES 데이터셋

In [31]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Processing...
Done!


In [ ]:
len(dataset)  # 600
dataset.num_classes  # 6
dataset.num_node_features  # 3

# 6개의 class(효소 종류), 
# 3개의 node feature(원자 종류, 결합도, 전하), 
# 600개의 graph(분자) 존재

3

In [40]:
# 600개 그래프에 접근
data = dataset[0]  # 첫 번째 그래프
print(data)
# 37개의 노드, 각 노드당 3개의 피처, 168/2=84개의 undirected edges 존재

data.is_undirected()  # True

Data(edge_index=[2, 168], x=[37, 3], y=[1])


True

In [41]:
train_dataset = dataset[:540]  # 540 graphs for training
test_dataset = dataset[540:]  # 60 graphs for testing
# train:test = 9:1

In [42]:
dataset = dataset.shuffle()
# split 전에 데이터셋이 이미 섞여 있는지 확실하지 않다면, 무작위로 섞는 것이 좋음

In [44]:
perm = torch.randperm(len(dataset))
dataset = dataset[perm]
# 위(dataset.shuffle())와 동일

Cora 데이터셋

(semi-supervised graph node classification를 위한 표준 벤치마크 데이터셋)

In [45]:
from torch_geometric.datasets import Planetoid

In [46]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [50]:
len(dataset)  # 1
dataset.num_classes  # 7
dataset.num_node_features  # 1433

1433

In [51]:
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [ ]:
data.is_undirected()  # True
data.train_mask.sum().item()  # 140
data.val_mask.sum().item()  # 500
data.test_mask.sum().item()  # 1000

# train_mask: 모델 학습에 사용되는 노드
# val_mask: 모델 검증에 사용되는 노드
# test_mask: 모델 테스트에 사용되는 노드

1000

#### Mini-batches

PyG achieves parallelization over a mini-batch by creating sparse block diagonal adjacency matrices (defined by 'edge_index') and concatenating feature and target matrices in the node dimension.

(1) 희소 블록 대각 인접 행렬('edge_index로 정의됨)를 생성하고, (2) node 차원에서 feature 행렬과 target 행렬을 연결 -> 미니배치에 대한 병렬화

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

from torch_geometric.utils import scatter

In [57]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for batch in loader:
    print(batch)
    print(batch.num_graphs)  # 32

    x = scatter(batch.x, batch.batch, dim=0, reduce='mean')
    # 각 그래프의 노드 차원에서 노드 특징 별 평균 계산
    # average node features in the node dimension for each graph individually
    print(x.size())  # torch.Size([32, 21])
    print(x)
    print("\n")

DataBatch(edge_index=[2, 3312], x=[859, 21], y=[32], batch=[859], ptr=[33])
32
torch.Size([32, 21])
tensor([[ 5.4167e+00,  1.1919e+01,  1.7002e+01,  2.5117e+00,  7.8433e-01,
          3.9592e+01,  2.0833e+00,  2.6667e+00,  6.6667e-01,  9.1667e-01,
          1.3333e+00,  3.1667e+00,  3.1667e+00,  1.3333e+00,  9.1667e-01,
          1.4167e+00,  3.3333e+00,  6.6667e-01,  2.5000e-01,  7.5000e-01,
          0.0000e+00],
        [ 7.0000e+00,  1.3867e+01,  2.4750e+01,  1.3790e+00,  1.1439e+00,
          5.6307e+01,  2.0690e+00,  3.1379e+00,  1.7931e+00,  2.0000e+00,
          2.1379e+00,  2.8621e+00,  3.0000e+00,  1.8966e+00,  2.1034e+00,
          1.7586e+00,  3.4483e+00,  1.7931e+00,  3.4483e-01,  6.5517e-01,
          0.0000e+00],
        [ 6.1429e+00,  1.2344e+01,  2.0450e+01, -4.7714e-01,  9.2571e-01,
          5.3457e+01,  2.7143e+00,  2.4286e+00,  1.0000e+00,  2.2857e+00,
          1.7143e+00,  2.1429e+00,  2.1429e+00,  1.7143e+00,  2.2857e+00,
          2.2857e+00,  2.8571e+00,  1.00

#### Data Transforms

ShapeNet 데이터셋

(17,000 3D shape point clouds and per point labels from 16 shape categories)

In [71]:
from torch_geometric.datasets import ShapeNet

import torch_geometric.transforms as T


In [72]:
dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])
# 스탠포트 대학교에서 제공하는 3D 모델 데이터셋인데, 스탠포드 연구실 서버 문제

dataset[0]
# >>> Data(pos=[2518, 3], y=[2518])

URLError: <urlopen error [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다>

In [ ]:
dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))
# point cloud에서 최근접 이웃 그래프를 생성하여 point cloud를 그래프로 변환

dataset[0]
# >>> Data(edge_index=[2, 15108], pos=[2518, 3], y=[2518])

In [ ]:
dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomJitter(0.01))
# transform: 데이터셋을 로드할 때마다 무작위로 점 위치에 작은 잡음을 추가/증가

dataset[0]
# >>> Data(edge_index=[2, 15108], pos=[2518, 3], y=[2518])

#### Learning Methods on Graphs

위에서 데이터 처리(data handling), 데이터 세트(datasets), 로더(loader), 변환(transforms)에 대한 내용 학습
-> 그래프 신경망 구성 (GCN layer 사용)

In [74]:
# load dataset (Cora)

from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)  # Cora()

Cora()


In [ ]:
# NOT need 'transforms' or 'dataloaders' for single graph datasets like Cora

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# Define a two-layer GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [79]:
model

# self.conv1 = GCNConv(dataset.num_node_features(=1433), 16)
# self.conv2 = GCNConv(16, dataset.num_classes(=7))

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [80]:
# Evaluate the model
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7960
